In [122]:
!pip install neptune-client

In [123]:
import neptune
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import re
sns.set(style="darkgrid")

project = neptune.init(
            api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiNGIzMTllNDQtZGVjMi00MjcxLThiOTAtYTcxNzJiNWI5NjkzIn0=",
            project_qualified_name="elangenhan/hcd-experiments"
        )

In [130]:
experiments = project.get_experiments()

logs_names_deprecated = ['dur', 'train_loss', 'train_acc', 'train_f1', 'train_roc_auc', 'train_precision', 'train_recall', 'test_acc' , 'test_f1' , 'test_roc_auc' , 'test_precision' , 'test_recall']
logs_names            = ['dur', 'train_loss', 'train_acc', 'train_f1', 'train_roc_auc', 'train_precision', 'train_recall', 'valid_acc', 'valid_f1', 'valid_roc_auc', 'valid_precision', 'valid_recall', 'valid_loss']


counter = 1
length = len(experiments)
metrics_df = pd.DataFrame(columns=['id', *logs_names])
for experiment in experiments[::-1]:
  print(f'{counter}/{length} - {experiment.id} \r')
  counter = counter + 1
  if experiment.state == 'succeeded' and experiment.get_parameters()['max_epochs'] and experiment.get_logs()['dur']['lastX'] == 30.0:
    if int(experiment.id.split('-')[1]) < 114:
      df = experiment.get_numeric_channels_values(*logs_names_deprecated)
      df = df.rename(columns={'test_acc': 'valid_acc', 'test_f1': 'valid_f1', 'test_roc_auc': 'valid_roc_auc', 'test_precision': 'valid_precision', 'test_recall': 'valid_recall'})
      df.insert(loc=12, column='valid_loss', value="NA")
    else:
      df = experiment.get_numeric_channels_values(*logs_names)

    # net_type
    net_type = experiment.get_parameters()["module"].split('\'')[1].split('.')[1].split('Pretrained')[0]
    df.insert(loc=13, column='net_type', value=net_type)

    # net_family
    match = re.match(r"([a-z]+)([0-9]+)", experiment.get_parameters()["module"].split('\'')[1].split('.')[1], re.I)
    if match: items = match.groups()
    df.insert(loc=13, column='net_family', value=items[0])

    # pretrained
    pretrained = "not pretrained"
    if "pretrained" in experiment.name.lower(): pretrained = "pretrained"
    df.insert(loc=14, column='pretrained', value=pretrained)

    # normalized
    normalized = "not normalized"
    if "normalized" in experiment.name.lower(): normalized = "normalized"
    df.insert(loc=15, column='normalized', value=normalized)

    # full_data
    full_data = ""
    if hasattr(experiment.get_logs(), 'test_name'):
      if "small" in experiment.get_logs()['test_name'].y:
        full_data = "partial data"
      else:
        full_data = "full data"
    else:
      if "small" in experiment.name.lower():
        full_data = "partial data"
      else:
        full_data = "full data"

    df.insert(loc=16, column='full_data', value=full_data)

    df.insert(loc=0, column='id', value=experiment.id)
    metrics_df = metrics_df.append(df, sort=True)

metrics_df.to_csv (r'./neptune_logs.csv', index = False, header=True)
print("done saving")

1/110 - HCDEX-305 
2/110 - HCDEX-304 
3/110 - HCDEX-303 
4/110 - HCDEX-302 
5/110 - HCDEX-301 
6/110 - HCDEX-300 
7/110 - HCDEX-299 
8/110 - HCDEX-298 
9/110 - HCDEX-297 
10/110 - HCDEX-296 
11/110 - HCDEX-294 
12/110 - HCDEX-293 
13/110 - HCDEX-292 
14/110 - HCDEX-291 
15/110 - HCDEX-290 
16/110 - HCDEX-289 
17/110 - HCDEX-288 
18/110 - HCDEX-287 
19/110 - HCDEX-286 
20/110 - HCDEX-285 
21/110 - HCDEX-284 
22/110 - HCDEX-283 
23/110 - HCDEX-282 
24/110 - HCDEX-281 
25/110 - HCDEX-280 
26/110 - HCDEX-279 
27/110 - HCDEX-278 
28/110 - HCDEX-277 
29/110 - HCDEX-276 
30/110 - HCDEX-275 
31/110 - HCDEX-174 
32/110 - HCDEX-172 
33/110 - HCDEX-167 
34/110 - HCDEX-158 
35/110 - HCDEX-157 
36/110 - HCDEX-156 
37/110 - HCDEX-152 
38/110 - HCDEX-147 
39/110 - HCDEX-146 
40/110 - HCDEX-145 
41/110 - HCDEX-144 
42/110 - HCDEX-143 
43/110 - HCDEX-142 
44/110 - HCDEX-141 
45/110 - HCDEX-140 
46/110 - HCDEX-139 
47/110 - HCDEX-138 
48/110 - HCDEX-137 
49/110 - HCDEX-136 
50/110 - HCDEX-135 
51/110 - 